<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=160px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Epidemiology 201</h1>
<h1>Social Networks</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import networkx as nx

import watermark

import epidemik
from epidemik import NetworkEpiModel

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 6f3f47aa98b5141fcddbadd07afeedaa91c4cfb7

numpy     : 1.26.4
watermark : 2.4.3
epidemik  : 0.0.6
pandas    : 2.1.4
networkx  : 3.1
matplotlib: 3.8.0



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## A toy network

In [4]:
G = nx.Graph()
G.add_edges_from([
    (1, 3),
    (3, 4),
    (4, 5),
    (4, 6),
    (4, 7),
    (4, 8),
    (8, 7),
    (8, 6),
    (6, 5),
    (6, 0),
    (6, 9),
    (5, 0),
    (5, 2)
])

The degree of each node is given by:

In [5]:
G.degree()

DegreeView({1: 1, 3: 2, 4: 5, 5: 4, 6: 5, 7: 2, 8: 3, 0: 2, 9: 1, 2: 1})

And we can easily add attributes to each node

In [6]:
nx.set_node_attributes(G, dict(zip(range(10), ('M', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F'))), 'Gender')
nx.set_node_attributes(G, dict(G.degree()), 'Degree')

And extract the attributes of each node into a DataFrame

In [7]:
nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
nodes.index.name='Node'
nodes.sort_index(inplace=True)

In [8]:
nodes

,Gender,Degree
Node,,
0,M,2
1,F,1
2,M,1
3,M,2
4,F,5
5,F,4
6,M,5
7,M,2
8,F,3


## Hubs

Let's define a hub as being the node with the largest degree of a network. We can find the largest degree of the network with

In [9]:
kmax = max([deg for node, deg in G.degree()])
kmax

5

And the nodes with that degree are:

In [10]:
hubs = [node for node, deg in G.degree() if deg == kmax]
hubs

[4, 6]

As expected.

## Contact tracing

If we assume that node 6 becomes infected, we can find who is at risk by checking his contacts:

In [11]:
contacts = list(G.neighbors(6))
contacts

[4, 8, 5, 0, 9]

And if we want to take the contact tracing a step further, we can also get the contacts of these nodes:

In [12]:
full_contacts = set(contacts)

for node in contacts:
    full_contacts |= set(G.neighbors(node))

In [13]:
contacts

[4, 8, 5, 0, 9]

In [14]:
full_contacts

{0, 2, 3, 4, 5, 6, 7, 8, 9}

This also illustrates the fact that 

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>